## Examining duplication Between Organisations

In our data we often recieve multiple data sources per dataset. unfortunately this leads to duplication of geometries and other data points in the datasets. this notebook looks to investigate identifying these duplications between organisations.

In [ ]:
# from download_data import download_dataset
# from data import get_entity_dataset, nrow
# from plot import plot_map, plot_issues_map
import spatialite
import pandas as pd
import geopandas as gpd
import os
import itertools
import shapely.wkt

import matplotlib.pyplot as plt
import time
import urllib

import numpy as np

pd.set_option("display.max_rows", 100)


In [ ]:
# if running on Colab, uncomment and run this line below too:
# !pip install mapclassify

### Functions

In [ ]:
def nrow(df):
    return print(f"No. of records in df: {len(df):,}")


def plot_issues_map(gdf:gpd.GeoDataFrame, entity_list, chloro_var, palette):

    if type(gdf) != gpd.GeoDataFrame:
        logging.error('input is not a GeodataFrame')
    
    base = gdf[gdf["entity"].isin(entity_list)].explore(
        column = chloro_var,  # make choropleth based on "BoroName" column
        cmap = palette,
        tooltip = False,
        popup = ["organisation_name", "entity", "name", "reference"],
        tiles = "CartoDB positron",  # use "CartoDB positron" tiles
        highlight = False,
        style_kwds = {
        "fillOpacity" : "0.1"
        }
    )
    
    return base

def get_all_organisations():
    params = urllib.parse.urlencode({
        "sql": f"""
        select organisation, name, entity as organisation_entity, statistical_geography
        from organisation
        """,
        "_size": "max"
        })
    url = f"https://datasette.planning.data.gov.uk/digital-land.csv?{params}"
    df = pd.read_csv(url)
    return df

### Data import

In [ ]:
# get LAD to LPA lookup from github
lookup_lad_lpa = pd.read_csv("https://github.com/digital-land/organisation-collection/raw/main/data/local-authority.csv",
                             usecols = ["entity", "local-authority-district", "local-planning-authority"])

lookup_lad_lpa.columns = ["organisation_entity", "LADCD", "LPACD"]

nrow(lookup_lad_lpa)
lookup_lad_lpa.head()

**Note on LAD to LPA mapping**   
Currently this [lookup file from github](https://github.com/digital-land/organisation-collection/raw/main/data/local-authority.csv) just records a 1:1 link between LADs and LPAs, but according to the ONS this relationship is actually 1:many. 
See [2020 lookup file](https://geoportal.statistics.gov.uk/datasets/ons::local-planning-authority-to-local-authority-district-april-2020-in-the-united-kingdom-lookup-1/about) and the example of Ryedale [`E07000167`], which is mapped to the following two LPAs:

* Ryedale LPA [`E60000061`]
* North York Moors National Park LPA [`E60000322`]

We need to agree some validation rules around this, i.e. can we expect Ryedale to submit data that might sit within either of these LPA areas, or for any London Boroughs to submit within the "London Legacy Development Corporation LPA" area?
But for simplicity's sake at the moment to get things up and running (as per Owen's advice), will test with existing 1:1 mapping and aim to develop logic once there is more clarity about multiple area handling.

The git lookup file also seems to be missing some areas, e.g. "Peak District National Park Authority" entity 405.

In [ ]:
lookup_org[lookup_org["LADCD"] == "E07000167"]

In [ ]:
# get org data from datasette
lookup_org = get_all_organisations()

# lookup_org["organisation_entity"] = lookup_org["organisation_entity"].astype(str)
lookup_org.columns = ["organisation", "organisation_name", "organisation_entity", "statistical_geography"]

# split out org type and join on LPA codes from LAD to LPA lookup
lookup_org["organisation_type"] = lookup_org["organisation"].apply(lambda x: x.split(":")[0])
lookup_org = lookup_org.merge(lookup_lad_lpa, how = "left", on = "organisation_entity")

nrow(lookup_org)
lookup_org.head()

In [ ]:
# check what types of org are missing the LPA code
nrow(lookup_org[lookup_org["LPACD"].isnull()])
lookup_org[lookup_org["LPACD"].isnull()].groupby("organisation_type").size()

In [ ]:
# LPA boundary data from planning.data.gov

LPA_boundary_df = pd.read_csv("https://files.planning.data.gov.uk/dataset/local-planning-authority.csv", 
                                  usecols = ["reference", "name", "geometry"])

LPA_boundary_df.columns = ["geometry", "name", "LPACD"]


# load geometry and create GDF
LPA_boundary_df['geometry'] = LPA_boundary_df['geometry'].apply(shapely.wkt.loads)
LPA_boundary_gdf = gpd.GeoDataFrame(LPA_boundary_df, geometry='geometry')

# Transform to ESPG:27700 for more interpretable area units
LPA_boundary_gdf.set_crs(epsg=4326, inplace=True)
LPA_boundary_gdf.to_crs(epsg=27700, inplace=True)

nrow(LPA_boundary_gdf)
LPA_boundary_gdf.head()


In [ ]:
# load conservation area entity dataset from planning.data.gov into geopandas and transform CRS to EPSG:27700

entity_df = pd.read_csv("https://files.planning.data.gov.uk/dataset/conservation-area.csv",
                            usecols = ["entity", "name", "organisation-entity", "reference", "geometry"])
            
# entity_df.head()
entity_df.columns = [x.replace("-", "_") for x in entity_df.columns]



# set entity to string, needed later to sort and remove duplicate self intersections
entity_df["entity"] = entity_df["entity"].astype(str)
# entity_df["organisation_entity"] = entity_df["organisation_entity"].astype(str)

# join organisation name and LPA codes from lookup
entity_df = entity_df.merge(
    lookup_org[["organisation_name", "organisation_type", "organisation_entity", "LPACD"]], 
    how = "left",
    on = "organisation_entity")

# load geometry and create GDF
entity_df['geometry'] = entity_df['geometry'].apply(shapely.wkt.loads)
entity_gdf = gpd.GeoDataFrame(entity_df, geometry='geometry')

# Transform to ESPG:27700 for more interpretable area units
entity_gdf.set_crs(epsg=4326, inplace=True)
entity_gdf.to_crs(epsg=27700, inplace=True)

# calculate area
entity_gdf["area"] = entity_gdf["geometry"].area

nrow(entity_gdf)
entity_gdf.head()

In [ ]:
# check of the organisations that we don't have an LPA code for
entity_df[entity_df["LPACD"].isnull()].groupby(["organisation_type", "organisation_name"]).size()

# Identifying geographical duplicates

## #1 - Intersection within organisation

In [ ]:
# Overlay all non-Heritage England entities
# [note - I was initially excluding Historic England from this, but don't see any reason to, changed below to include all]
LPA_LPA_join = gpd.overlay(
    entity_gdf, entity_gdf,
    # entity_gdf[entity_gdf["organisation_entity"] != 16],
    # entity_gdf[entity_gdf["organisation_entity"] != 16],
    how = "intersection", keep_geom_type=False 
)

# remove entity self-intersections and intersections across organisations
LPA_LPA_join = LPA_LPA_join[(LPA_LPA_join["organisation_entity_1"] == LPA_LPA_join["organisation_entity_2"]) &
             (LPA_LPA_join["entity_1"] != LPA_LPA_join["entity_2"])]

# each intersection will be in there twice because we're joining the same dataset 
# (e.g. polygon1-polygon2 and polygon2-polygon1), so remove these
LPA_LPA_join["entity_join"] = LPA_LPA_join.apply(lambda x: '-'.join(sorted(x[["entity_1", "entity_2"]])), axis=1)
LPA_LPA_join.drop_duplicates(subset="entity_join", inplace = True) #Drop them by name

# calculate overlap %'s

LPA_LPA_join["area_intersection"] = LPA_LPA_join["geometry"].area

# LPA_LPA_join["p_pct_intersect"] = LPA_LPA_join["area_intersection"] / LPA_LPA_join["area_1"]
# LPA_LPA_join["pct_intersection"] = LPA_LPA_join["area_intersection"] / (LPA_LPA_join["area_1"] + LPA_LPA_join["area_2"] - LPA_LPA_join["area_intersection"])
# LPA_LPA_join["s_pct_intersect"] = LPA_LPA_join["area_intersection"] / LPA_LPA_join["area_2"]

# intersection area as % of smallest primary or secondary area
LPA_LPA_join["pct_min_intersection"] = LPA_LPA_join["area_intersection"] / LPA_LPA_join[["area_1", "area_2"]].min(axis = 1)

nrow(LPA_LPA_join)
LPA_LPA_join.head()

In [ ]:
# quick check of distribution to check how many are edges vs. major overlaps
plt.hist(LPA_LPA_join["pct_min_intersection"], bins=50);

In [ ]:
# how many entities with a greater than 10% intersection?
nrow(LPA_LPA_join[(LPA_LPA_join["pct_min_intersection"] > 0.5)])

# LPA_LPA_join[(LPA_LPA_join["pct_min_intersection"] > 0.1)].sort_values("pct_min_intersection", ascending=False)

In [ ]:
# count by organisation of entities with intersections > 10%
LPA_LPA_join[(LPA_LPA_join["pct_min_intersection"] > 0.1)].groupby(["organisation_name_1"]).size().sort_values(ascending = False)

In [ ]:
LPA_LPA_join[(LPA_LPA_join["organisation_name_1"] == "Historic England")].sort_values("pct_min_intersection", ascending = False).head()

**notes from run through with Swati**

solution - go back to LPA
possible explanation - data is coming from different endpoint, and first one is not retired. Need to rule this out before we go back to LPA.

When new endpoint is added, we want to keep both. Want to keep record of data over time. Platform should only present latest version.
Need to understand entity creation process a little bit more to understand how geo duplicates could get made - talk to Kena.

In [ ]:
# inspect example
# plot_issues_map(entity_gdf, ["44005062", "44002577"], "name", "Accent")
plot_issues_map(entity_gdf, ["44000171", "44000170"], "name", "Accent")


In [ ]:
# inspect example
plot_issues_map(entity_gdf, ["44008830", "44006848"], "name", "Accent")


## #2 - Intersection across different LPAs

In [ ]:
# Overlay all non-Heritage England entities
LPA_cross_join = gpd.overlay(
    entity_gdf[entity_gdf["organisation_entity"] != 16],
    entity_gdf[entity_gdf["organisation_entity"] != 16],
    how = "intersection", keep_geom_type=False 
)

# filter to join across organisations and entities
LPA_cross_join = LPA_cross_join[(LPA_cross_join["organisation_entity_1"] != LPA_cross_join["organisation_entity_2"]) &
             (LPA_cross_join["entity_1"] != LPA_cross_join["entity_2"])]

# each intersection will be in there twice because we're joining the same dataset 
# (e.g. polygon1-polygon2 and polygon2-polygon1), so remove these
LPA_cross_join["entity_join"] = LPA_cross_join.apply(lambda x: '-'.join(sorted(x[["entity_1", "entity_2"]])), axis=1)
LPA_cross_join.drop_duplicates(subset="entity_join", inplace = True) #Drop them by name

# # calculate overlap %'s

LPA_cross_join["area_intersection"] = LPA_cross_join["geometry"].area

# # LPA_LPA_join["p_pct_intersect"] = LPA_LPA_join["area_intersection"] / LPA_LPA_join["area_1"]
# # LPA_LPA_join["pct_intersection"] = LPA_LPA_join["area_intersection"] / (LPA_LPA_join["area_1"] + LPA_LPA_join["area_2"] - LPA_LPA_join["area_intersection"])
# # LPA_LPA_join["s_pct_intersect"] = LPA_LPA_join["area_intersection"] / LPA_LPA_join["area_2"]

# intersection area as % of smallest primary or secondary area
LPA_cross_join["pct_min_intersection"] = LPA_cross_join["area_intersection"] / LPA_cross_join[["area_1", "area_2"]].min(axis = 1)

nrow(LPA_cross_join)
LPA_cross_join.head()

In [ ]:
# Look at distribution to check how many are edges vs. major overlaps
plt.hist(LPA_cross_join["pct_min_intersection"], bins=50);

In [ ]:
# how many entities which have issues of intersection > 10%? 
LPA_cross_join[(LPA_cross_join["pct_min_intersection"] > 0.1)]

In [ ]:
e = "44009059"
ents = LPA_cross_join[(LPA_cross_join["entity_1"] == e)][["entity_1", "entity_2"]].iloc[0, :].values

plot_issues_map(entity_gdf, ents, "organisation_name", "Accent")

## #2.b - Beyond-border data

In [ ]:
# List LPA codes from entity df and check they're all in the LPA gdf
lpa_list = entity_df["LPACD"][entity_df["LPACD"].notnull()].drop_duplicates().to_list()

# check every one of our entity LPAs is in the LPA gdf
print(len(lpa_list))
nrow(LPA_boundary_gdf[LPA_boundary_gdf["LPACD"].isin(lpa_list)])

In [ ]:
geogs_out_entities = []

# loop through LPA codes and for each check whether any conservation areas with that code don't intersect at all with the LPA boundary
for lpa_code in lpa_list:

    cons_areas = entity_gdf[entity_gdf["LPACD"] == lpa_code]
    cons_areas_intersect = cons_areas.geometry.intersects(LPA_boundary_gdf[LPA_boundary_gdf["LPACD"] == lpa_code].iloc[0].geometry)

    # add areas which don't intersect to the list
    geogs_out_entities.extend(cons_areas.loc[~cons_areas_intersect]["entity"].to_list())


entity_outside_LPA_df = entity_df[entity_df["entity"].isin(geogs_out_entities)]

# list of LPAs with entities outside them
LPAs_with_bads = entity_outside_LPA_df["LPACD"].drop_duplicates().to_list()

print(f"No. of bad entities found: {len(entity_outside_LPA_df):,}")
entity_outside_LPA_df.groupby("organisation_name").size()


In [ ]:
# map for LPA with entities outside it

LPA_code = LADs_with_bads[0]
bad_ents = entity_outside_LPA_df["entity"][entity_outside_LPA_df["LPACD"] == LPA_code]


map_entities = entity_gdf[entity_gdf["entity"].isin(bad_ents)].explore(
        # column = chloro_var,  # make choropleth based on "BoroName" column
        # cmap = palette,
    color = "red",
        # tooltip = False,
        # popup = ["organisation_name", "entity", "name", "reference"],
        tiles = "CartoDB positron",  # use "CartoDB positron" tiles
        highlight = False,
        style_kwds = {
        "fillOpacity" : "0.1"
        }
)

LPA_boundary_gdf[LPA_boundary_gdf["LPACD"] == LPA_code].explore(
    m = map_entities,
    color = "blue",
        style_kwds = {
        "fillOpacity" : "0"
        }
)

In [ ]:
# map for LPA with entities outside it

LPA_code = LADs_with_bads[2]
bad_ents = entity_outside_LPA_df["entity"][entity_outside_LPA_df["LPACD"] == LPA_code]


map_entities = entity_gdf[entity_gdf["entity"].isin(bad_ents)].explore(
        # column = chloro_var,  # make choropleth based on "BoroName" column
        # cmap = palette,
    color = "red",
        # tooltip = False,
        # popup = ["organisation_name", "entity", "name", "reference"],
        tiles = "CartoDB positron",  # use "CartoDB positron" tiles
        highlight = False,
        style_kwds = {
        "fillOpacity" : "0.1"
        }
)

LPA_boundary_gdf[LPA_boundary_gdf["LPACD"] == LPA_code].explore(
    m = map_entities,
    color = "blue",
        style_kwds = {
        "fillOpacity" : "0"
        }
)

## Issue #3 LPA => Historic England intersections

In [ ]:
# start_time = time.time()

LPA_HE_join = gpd.overlay(
    entity_gdf[entity_gdf["organisation_entity"] != 16],
    entity_gdf[entity_gdf["organisation_entity"] == 16],
    how = "intersection", keep_geom_type=False
)

LPA_HE_join["area_intersection"] = LPA_HE_join["geometry"].area

LPA_HE_join["p_pct_intersect"] = LPA_HE_join["area_intersection"] / LPA_HE_join["area_1"]
LPA_HE_join["pct_intersection"] = LPA_HE_join["area_intersection"] / (LPA_HE_join["area_1"] + LPA_HE_join["area_2"] - LPA_HE_join["area_intersection"])
LPA_HE_join["s_pct_intersect"] = LPA_HE_join["area_intersection"] / LPA_HE_join["area_2"]


# intersection area as % of smallest primary or secondary area
LPA_HE_join["pct_min_intersection"] = LPA_HE_join["area_intersection"] / LPA_HE_join[["area_1", "area_2"]].min(axis = 1)


# end_time = time.time()

# elapsed_time = (end_time - start_time) 
# print(f"Elapsed time: {elapsed_time:.2f} ")

nrow(LPA_HE_join)
LPA_HE_join.head()

In [ ]:
# plot the issues by the amount the two entities which make up each issue intersect each other
# this is useful to start to define categories for the types of issues they represent

fig = plt.figure()
plt.grid()
plt.scatter(LPA_HE_join["p_pct_intersect"], LPA_HE_join["s_pct_intersect"], s = 8, alpha=0.6)
fig.suptitle('Entity intersection %s', fontsize=14)
plt.xlabel('% of LPA entity intersected', fontsize=10)
plt.ylabel('% of Historic England entity intersected', fontsize=10)

By the number of points on the far right of the chart we can see that there are a lot of LPA entities which are entirely or almost entirely contained within an HE entity, but how closely the HE area matches varies from not at all to almost exactly.

Bottom left is a cluster of tiny edge intersections, and there are a small number of instances where HE entities are contained within LPA ones.

In [ ]:
# flag issue types - defined to pick up main issue clusters on chart above and using a 90% or 10% intersection cutoffs

LPA_HE_join["issue_type"] = np.select(
    [
        (LPA_HE_join["p_pct_intersect"] >= 0.9) & (LPA_HE_join["s_pct_intersect"] >= 0.9),
        (LPA_HE_join["p_pct_intersect"] <= 0.1) & (LPA_HE_join["s_pct_intersect"] <= 0.1),
        (LPA_HE_join["p_pct_intersect"] >= 0.9),
        (LPA_HE_join["s_pct_intersect"] >= 0.9)
    ],
    [
        "LPA and HE cover each other", "edge intersection", "LPA covered by HE", "LPA covers HE"
    ],
    default = "-"
)

In [ ]:
# LPA_HE_join[(LPA_HE_join["pct_intersection"] >= 0.9)].sort_values("pct_intersection")
# LPA_HE_join[(LPA_HE_join["issue_type"] == "LPA covers HE")].sort_values("pct_intersection")

In [ ]:
# count of issue types (where cover is defined as >=90% intersection, and edge as <=10%)
LPA_HE_join.groupby(["issue_type"]).size()

In [ ]:
# LPAs with most non-intersection issues
LPA_HE_join[(LPA_HE_join["issue_type"] != "edge intersection")].groupby(["organisation_name_1"]).size().sort_values(ascending = False).head(15)

### Issue examples by type

#### LPA and HE cover each other (almost perfect matches)

- need to get to the bottom of authority here, who can create conservation areas
- could we just switch off HE conservation areas 

In [ ]:
e = "44008960"
ents = LPA_HE_join[(LPA_HE_join["entity_1"] == e)][["entity_1", "entity_2"]].iloc[0, :].values

plot_issues_map(entity_gdf, ents, "organisation_name", "Accent")


#### LPA covered by HE

In [ ]:
# LPA_HE_join[(LPA_HE_join["issue_type"] == "LPA covered by HE")].sort_values("pct_min_intersection", ascending = False).head()

In [ ]:
e = "44009177"
ents = LPA_HE_join[(LPA_HE_join["entity_1"] == e)][["entity_1", "entity_2"]].iloc[0, :].values

plot_issues_map(entity_gdf, ents, "organisation_name", "Accent")


#### LPA covers HE

In [ ]:
e = "44009160"
ents = LPA_HE_join[(LPA_HE_join["entity_1"] == e)][["entity_1", "entity_2"]].iloc[0, :].values

plot_issues_map(entity_gdf, ents, "organisation_name", "Accent")

#### Edge intersection

In [ ]:
e = "44006512"
ents = LPA_HE_join[(LPA_HE_join["entity_1"] == e)][["entity_1", "entity_2"]].iloc[0, :].values

plot_issues_map(entity_gdf, ents, "organisation_name", "Accent")

#### Entities with multiple issues

In [ ]:
# LPA_HE_join[(LPA_HE_join["entity_1"] == "44006512")]

entity_count = LPA_HE_join.groupby(["entity_1"]).size().reset_index()
entity_count.columns = ["entity_1", "count"]
entity_count[entity_count["count"] > 1].sort_values('count', ascending = False)

In [ ]:
t = LPA_HE_join[LPA_HE_join["entity_1"] == "44009090"]

# grab all entities that have an issue with  44009090
te = np.concatenate((
    t["entity_1"].drop_duplicates().values,
    t["entity_2"].drop_duplicates().values
))

plot_issues_map(entity_gdf, te, "organisation_name", "Accent")


#### Entities with non-classified issues

In [ ]:
# these are really just entities which have overlaps > 10% but less than 90% in one form 
LPA_HE_join[(LPA_HE_join["issue_type"] == "-")].sort_values("pct_min_intersection", ascending = False).head()

In [ ]:
# looking at entity re-directs
entity_df[entity_df["entity"].isin(["44000549", "44008664"])]

## Questions to resolve
* how to find endpoint / resource for each entity?
* what existing issues / replacements have been documented for the dataset?
* which entity takes precedence? Oldest / newest?
* what threshold to set for removing duplicates?
* how to extract data required for updating through lookups file
* how to replicate this check in endpoint checker with a new dataset

**notes from run-through with Swati**   
feed in LPA boundaries here to make sure we contact the right LPA - change this query to use LPA boundaries.
check with Carlos for how it's been done for brownfield